# Application: Creating a Telco Mobility Index

1. Add total travel distance
2. Add radius of gyration
3. Add activity entropy

In [1]:
import os
os.chdir("../")

import random
import shapely
import pendulum
import pandas as pd
from sds4gdsp.processor import convert_cel_to_point
from sds4gdsp.plotter import get_route_fig, load_images, plot_images

In [2]:
filepath_subscribers = "data/fake_subscribers.csv"
fake_subscribers = pd.read_csv(filepath_subscribers)
fake_subscribers.sample(5)

,uid,gender,age,name,chi_indicator,ewallet_user_indicator
31,glo-sub-032,female,60,Kathleen Miller,0,N
72,glo-sub-073,female,39,Jamie Erickson,0,N
58,glo-sub-059,female,19,Barbara Martinez,0,N
47,glo-sub-048,female,51,Cassidy Sharp,0,Y
83,glo-sub-084,male,20,Donald Munoz,0,N


In [3]:
filepath_cellsites = "data/fake_cellsites.csv"
fake_cellsites = pd.read_csv(filepath_cellsites)
fake_cellsites.sample(5)

,uid,coords
19,glo-cel-020,POINT (121.0454228 14.462994)
86,glo-cel-087,POINT (121.0751054 14.5445315)
57,glo-cel-058,POINT (121.05919 14.5303819)
64,glo-cel-065,POINT (121.0624397 14.5289908)
52,glo-cel-053,POINT (121.0581692 14.5056112)


In [4]:
filepath_transactions = "data/fake_transactions.csv"
fake_transactions = pd.read_csv(filepath_transactions)

In [5]:
min_date = pendulum.parse(fake_transactions.transaction_dt.min(), exact=True)
max_date = pendulum.parse(fake_transactions.transaction_dt.max(), exact=True)
period = pendulum.period(min_date, max_date)

In [6]:
sample_dt = str(random.sample(list(period), 1)[0])
filter_dt = fake_transactions.transaction_dt == sample_dt
sample_sub = fake_subscribers.uid.sample(1).item()
filter_sub = fake_transactions.sub_id == sample_sub
fake_transactions.loc[filter_sub&filter_dt]

,uid,sub_id,cel_id,transaction_dt,transaction_hr
28071,glo-txn-28072,glo-sub-084,glo-cel-092,2023-06-22,5
28072,glo-txn-28073,glo-sub-084,glo-cel-087,2023-06-22,7
28073,glo-txn-28074,glo-sub-084,glo-cel-095,2023-06-22,8
28074,glo-txn-28075,glo-sub-084,glo-cel-087,2023-06-22,11
28075,glo-txn-28076,glo-sub-084,glo-cel-092,2023-06-22,13
28076,glo-txn-28077,glo-sub-084,glo-cel-096,2023-06-22,14
28077,glo-txn-28078,glo-sub-084,glo-cel-095,2023-06-22,15
28078,glo-txn-28079,glo-sub-084,glo-cel-093,2023-06-22,17
28079,glo-txn-28080,glo-sub-084,glo-cel-095,2023-06-22,18
28080,glo-txn-28081,glo-sub-084,glo-cel-096,2023-06-22,19


In [ ]:
# day in a life of a sub
d = fake_transactions.copy()
sample_sub = d.sample(1).sub_id.item()
d = d.loc[d.sub_id==sample_sub]
days = d.transaction_dt.unique().tolist()
route_figs = []
for sample_day in days:
    sites = fake_transactions\
        .loc[fake_transactions.sub_id==sample_sub]\
        .loc[fake_transactions.transaction_dt==sample_day]\
        .cel_id.tolist()
    points = list(map(lambda z: convert_cel_to_point(z, fake_cellsites), sites))
    r = shapely.geometry.LineString(points)
    route_figs.append(get_route_fig(r))
for idx, fig in enumerate(route_figs):
    fname = "sample/{}_tmp.jpg".format(idx+1)
    fig.savefig(fname)
imgs = load_images("sample")
plot_images(imgs)